In [1]:
import sys
sys.path.append("../")
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from utils.inkml2img import convert_dir
from tqdm.auto import tqdm
from data.dataset import Im2LatexDataset
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from model.vit import ViT
import lightning as L
import wandb
from transformers import PreTrainedTokenizerFast
from tokenizers import Tokenizer, pre_tokenizers
from tokenizers.models import BPE
from model.decoder import Decoder, DecoderAttention, DecoderTransformerBlock
from model.model import Img2MathModel


In [2]:

img_dims = [256, 256]
data = Im2LatexDataset(path_to_data="../data/",
                       tokenizer="../data/tokenizer.json", img_dims=img_dims, batch_size=2, device=torch.device('cpu'))
batch = next(iter(data.train))
img, label = batch
vocab_size = len(data.tokenizer.get_vocab())
model = Img2MathModel.load_from_checkpoint('../train/img2math/362edsd8/checkpoints/epoch=9-step=65180.ckpt', map_location=torch.device('cpu'))
model(img)
#model.eval()
#logits, _ = model(img)
#torch.argmax(logits)
#val_imgs = img
#model.training_step(batch, 1)

/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/utilities/migration/utils.py:51: PossibleUserWarning: The loaded checkpoint was produced with Lightning v2.0.4, which is newer than your current Lightning version: v2.0.3
  rank_zero_warn(


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│    5 img, label = batch                                                                          │
│    6 vocab_size = len(data.tokenizer.get_vocab())                                                │
│    7 model = Img2MathModel.load_from_checkpoint('../train/img2math/362edsd8/checkpoints/epoch    │
│ ❱  8 model(img)                                                                                  │
│    9 #model.eval()                                                                               │
│   10 #logits, _ = model(img)                                                                     │
│   11 #torch.argmax(logits)                                                                       │
│                                                                                                  │
│ /Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/torch/nn/modules/module.py:15 │
│ 01 in _call_impl                                                                                 │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                                                           │
│                                                                                                  │
│ /Users/dylanh/Projects/img2math/model/notebooks/../model/model.py:38 in forward                  │
│                                                                                                  │
│   35 │   │                                                                                       │
│   36 │   │   for i in range(self.block_size):                                                    │
│   37 │   │   │   logits = self.decoder(pred_sequence, image_encodings)                           │
│ ❱ 38 │   │   │   logits = logits.unsqueeze(1).expand(-1, 2160, -1)                               │
│   39 │   │   │   pred = torch.argmax(logits, dim=-1) # (B, 1)                                    │
│   40 │   │   │                                                                                   │
│   41 │   │   │   pred_sequence = torch.cat((pred_sequence, pred), dim=-1)                        │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
RuntimeError: expand(torch.FloatTensor{[2, 1, 1, 2160]}, size=[-1, 2160, -1]): the number of sizes provided (3) 
must be greater or equal to the number of dimensions in the tensor (4)

In [3]:
def decode(tokenizer, sequence):
    dec = [tokenizer.decode(tok) for tok in sequence]
    return ''.join([detok.replace('Ġ', ' ') for detok in dec])

#preds = [model.generate(img) for img in val_imgs]
tokenizer = PreTrainedTokenizerFast(tokenizer_file='../data/tokenizer.json', padding_side='left')

seq = model.generate(val_imgs[0])
decode(tokenizer, seq)



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│    5 #preds = [model.generate(img) for img in val_imgs]                                          │
│    6 tokenizer = PreTrainedTokenizerFast(tokenizer_file='../data/tokenizer.json', padding_sid    │
│    7                                                                                             │
│ ❱  8 seq = model.generate(val_imgs[0])                                                           │
│    9 decode(tokenizer, seq)                                                                      │
│   10                                                                                             │
│   11                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'val_imgs' is not defined

In [3]:
TOKENIZER_FILE = '../data/tokenizer.json'

def decode(tokenizer, sequence):
    dec = [tokenizer.decode(tok) for tok in sequence]
    return ''.join([detok.replace('Ġ', ' ') for detok in dec])

class ImagePredictionLogger(L.Callback):
    
    def __init__(self, val_samples, num_samples=10):
        super().__init__()
        self.tokenizer = PreTrainedTokenizerFast(tokenizer_file=TOKENIZER_FILE, padding_side='left')
        self.val_imgs, self.val_labels = val_samples
        self.val_imgs = self.val_imgs[:num_samples]
        self.val_labels = [decode(self.tokenizer, label) for label in self.val_labels['input_ids'][:num_samples]]


    def on_validation_epoch_end(self, trainer, pl_module):
        val_imgs = self.val_imgs.to(device=pl_module.device)

        pred_tokens = [pl_module.generate(x) for x in val_imgs]
        preds = [decode(self.tokenizer, pred) for pred in pred_tokens]

        trainer.logger.experiment.log({
            "examples": [wandb.Image(x, caption=f"Pred:{pred}, Label:{y}")
                         for x, pred, y in zip(val_imgs, preds, self.val_labels)],
            "global_step": trainer.global_step
        })

logger = WandbLogger(project='img2math')

trainer = L.Trainer(limit_train_batches=10, max_epochs=1, log_every_n_steps=10, deterministic=True,
                    logger=logger, accelerator='mps', callbacks=[ImagePredictionLogger(batch)])

trainer.fit(model, data.train, data.test)
wandb.finish()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: baolong. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params
------------------------------------
0 | encoder | ViT     | 8.9 M 
1 | decoder | Decoder | 14.9 M
------------------------------------
23.9 M    Trainable params
0         Non-trainable params
23.9 M    Total params
95.443    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


epoch,▁▁
global_step,▁█
train/loss,▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val/loss_epoch,▁
val/loss_step,▂▄▃▃▃▂▃▃▆▂▄▂▂▂▃▃█▁▂▂▁▃▄▃▂▆▂▂▁▃▃▂▄▃▃▂▇▃▃▁
epoch,0
global_step,10
train/loss,0.00864
trainer/global_step,9
val/loss_epoch,0.44301


torch.Size([1])